# 单轴调试

In [ ]:
import os
import sys
# 添加pcan_cybergear库的路径
sys.path.append(os.path.join("..", "cybergear"))

from pcan_cybergear import CANMotorController
import can
import logging
# Initialize logging
logging.basicConfig(level=logging.WARNING, format='%(asctime)s - %(levelname)s - %(message)s')

In [ ]:
# Connect to the CAN bus with 1 Mbit/s bitrate
bus = can.interface.Bus(bustype="pcan", channel="PCAN_USBBUS1", bitrate=1000000)
motor1 = CANMotorController(bus, motor_id=101, main_can_id=254)
motor2 = CANMotorController(bus, motor_id=102, main_can_id=254)
motor3 = CANMotorController(bus, motor_id=103, main_can_id=254)
motor4 = CANMotorController(bus, motor_id=104, main_can_id=254)

In [ ]:
motor = motor2

In [ ]:
motor.write_param_table("limit_cur", 1.5)

In [ ]:
motor.write_single_param("cur_filt_gain", 0.1)

In [ ]:
motor.write_param_table("loc_kp", 8)

In [ ]:
motor.write_param_table("spd_kp", 2)

In [ ]:
motor.write_param_table("spd_ki", 0.03)

In [ ]:
motor.write_param_table("cur_kp", 0.0125) # 默认0.125

In [ ]:
motor.write_param_table("cur_ki", 0.1) # 默认0.0158

In [ ]:
motor.write_param_table("spd_filt_gain", 0.1)

## 停止

In [ ]:
motor.disable()
motor.set_0_pos()

# 位置模式

In [ ]:
motor.set_run_mode(motor.RunModes.POSITION_MODE)

In [ ]:
motor.enable()

In [ ]:
import time
motor.write_single_param("loc_ref", value=-0.5)
time.sleep(1.5)
motor.write_single_param("loc_ref", value=0.5)
time.sleep(1.5)
motor.write_single_param("loc_ref", value=0)

In [ ]:
motor.write_single_param("limit_spd", value=4)

In [ ]:
motor.write_single_param("loc_ref", value=0)

In [ ]:
motor.set_motor_position_control(1, 0)

In [ ]:
motor.disable()
motor.set_0_pos()

# 运控模式

In [ ]:
motor.set_run_mode(motor.RunModes.CONTROL_MODE)

In [ ]:
motor.enable()

In [ ]:
motor.send_motor_control_command(torque=0, target_angle=0, target_velocity=0, Kp=1, Kd=0)

In [ ]:
motor.disable()
motor.set_0_pos()

# 速度模式

In [ ]:
motor.set_run_mode(motor.RunModes.SPEED_MODE)

In [ ]:
motor.enable()
motor.write_single_param("spd_ref", value=0)

In [ ]:
motor.disable()
motor.set_0_pos()

In [ ]:
motor.write_single_param("spd_ref", value=1)

In [ ]:
import time
start_time = time.time()
test_time = 15
speed = 2
motor.enable()
while True:
    elapsed_time = time.time() - start_time
    if elapsed_time >= 10:
        logging.warning(f"{test_time} seconds have passed. End test")
        motor.set_0_pos()
        motor.disable()
        break
    _, pos, vel, torque_ext = motor.write_single_param("spd_ref", value=speed)
    if abs(torque_ext) > 0.2:
        speed = -speed
        motor.write_single_param("spd_ref", value=speed)
        logging.warning(f"reverse on torque_ext: {torque_ext:.2f}, vel:{vel:.2f}")
    time.sleep(0.01)

# 电流模式

In [ ]:
motor.set_run_mode(motor.RunModes.CURRENT_MODE)

In [ ]:
# iq_ref
motor.enable()
motor.write_single_param("iq_ref", value=0)

In [ ]:
motor.write_single_param("iq_ref", value=0.2)

In [ ]:
motor.disable()
motor.set_0_pos()